In [1]:
with open('chatgpt_strong_awry_utterances.txt', 'r') as file:
    all_lines = file.readlines()
prompts = {}
for line in all_lines:
    id, prompt = line.strip().split(":=:=:=:=")
    prompts[id] = prompt

In [10]:
for id in prompts:

    if len(prompts[id]) < 15:
        # print(prompts[id])
        prompts[id]

In [18]:
import json
from convokit import Forecaster, download, Corpus, Utterance, Speaker, Conversation

corpus = Corpus(utterances=[])
speaker = Speaker()
with open('strong_awry_utterances.json', 'r') as file:
    data = json.load(file)
with open('chatgpt_strong_awry_utterances.txt', 'r') as file:
    all_lines = file.readlines()
prompts = {}
for line in all_lines:
    id, prompt = line.strip().split(":=:=:=:=")
    prompts[id] = prompt
all_new_utterances = []
for conv_id in data:
    utt_id = conv_id.split("_")[1]
    if prompts[conv_id]:
        new_convo = Conversation(owner=corpus, id=conv_id, meta={'has_removed_comment': False})
        prompt_utterance = Utterance(id=conv_id,
                            speaker=speaker,
                            conversation_id=conv_id,
                            reply_to=None,
                            text=prompts[conv_id],
                            timestamp=0,
                            )
        reply_utterance = Utterance(id=utt_id,
                            speaker=speaker,
                            conversation_id=conv_id,
                            reply_to=conv_id,
                            text=data[conv_id][-1]['text'],
                            timestamp=1,
                            )
        all_new_utterances.append(prompt_utterance)
        all_new_utterances.append(reply_utterance)
corpus.add_utterances(all_new_utterances)

In [23]:
count_convo = 0
for convo in corpus.iter_conversations():
    assert convo.meta['has_removed_comment'] == False
    assert len(convo.get_chronological_utterance_list()) == 2
    count_convo += 1
    for utt in convo.iter_utterances():
        print(utt)
    break
print(count_convo)    

Utterance(id: 'cus26gy_cus2wdb', conversation_id: cus26gy_cus2wdb, reply-to: None, speaker: Speaker(id: None, vectors: [], meta: ConvoKitMeta({})), timestamp: 0, text: 'How do you think the current economic system can be restructured to better support individuals pursuing careers focused on humanitarianism and activism?', vectors: [], meta: ConvoKitMeta({}))
Utterance(id: 'cus2wdb', conversation_id: cus26gy_cus2wdb, reply-to: cus26gy_cus2wdb, speaker: Speaker(id: None, vectors: [], meta: ConvoKitMeta({})), timestamp: 1, text: "&gt; Almost all transactions are such [Any transaction in which both sides benefit is an increase in welfare].\n\nThat's not true in this point in time. You are incredibly gullible if you think otherwise. Most transactions are thefts. \n", vectors: [], meta: ConvoKitMeta({}))
1
